In [1]:
from dotenv import load_dotenv
load_dotenv()
from langchain_google_genai import ChatGoogleGenerativeAI

llm = ChatGoogleGenerativeAI(model="gemini-2.5-flash")

In [2]:
SYSTEM_PROMPT = """
You are the Router for a restaurant assistant. Classify the user's message into exactly one intent:
- menu: questions about food/menu items, categories, veg/non-veg, ingredients, allergens, prices, recommendations, ratings, combos.
- about: questions about the restaurant itself: name, location/address, opening/closing hours, contact/phone, story/vision/mission, ambience.
- faqs: policies/amenities: parking, Wi-Fi, payment methods, pet/kid friendliness, wheelchair access, smoking area, delivery partners, refunds/returns, general facilities.
- escalate: requests to place/modify/cancel orders, make reservations/table bookings, payment/transaction issues, urgent complaints requiring staff.
- none: greetings, thanks, small talk, jokes, or generic conversation you can handle directly without tools.

Rules:
- Prefer escalate if the user asks to "book", "reserve", "order", "pay", "cancel", or similar operational requests.
- Prefer menu when the focus is food details, even if price/rating is mentioned.
- Prefer about for hours, address, contact, or the restaurant’s story.
- Prefer faqs for policies and facilities.
- Use none for chit-chat (hi/hello/thanks/ok/lol/good night) or general praise.

Return ONLY JSON in one line:
{"intent":"menu|about|faqs|escalate|none","confidence":0.0-1.0,"rationale":"<short reason>"}
Keep rationale under 12 words.
"""


In [3]:
def classify_intent_minimal(user_text: str) -> str:
    """
    Minimal version:
    - Sends system prompt + user text to the LLM
    - Returns the LLM's raw response (a one-line JSON string per the prompt)
    - No parsing, no guards, no exception handling
    """
    messages = [
        {"role": "system", "content": SYSTEM_PROMPT},
        {"role": "user", "content": f"User: {user_text}\nReturn only the JSON."},
    ]
    resp = llm.invoke(messages)
    return resp.content.strip()


In [4]:
# demo
if __name__ == "__main__":
    tests = [
        "hi there",
        "do you have veg pizzas under 500?",
        "what time do you open tomorrow?",
        "is there parking and wifi?",
        "can you book a table for 7pm?",
        "tell me about your founder's story",
    ]
    for t in tests:
        print(t, "->", classify_intent_minimal(t))

hi there -> {"intent":"none","confidence":1.0,"rationale":"User is providing a general greeting."}
do you have veg pizzas under 500? -> {"intent":"menu","confidence":1.0,"rationale":"User asks about specific menu items (veg pizzas) and price."}
what time do you open tomorrow? -> {"intent":"about","confidence":1.0,"rationale":"User asks about opening hours, which is restaurant information."}
is there parking and wifi? -> {"intent":"faqs","confidence":1.0,"rationale":"User asks about parking and Wi-Fi, which are restaurant facilities."}
can you book a table for 7pm? -> {"intent":"escalate","confidence":1.0,"rationale":"User wants to book a table, requires staff intervention."}
tell me about your founder's story -> {"intent":"about","confidence":1.0,"rationale":"User asks about the founder's story, related to the restaurant's history."}
